In [1]:
import os

import yaml
import json
import re

import process
from sklearn.feature_extraction.text import TfidfVectorizer

from importlib import reload
reload(process)

import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pickle

In [13]:
curr_dir = os.getcwd()
rasa_dir = os.path.join(curr_dir, '../rasa')
intents_filepath = os.path.join(rasa_dir, 'data/nlu/intents.yml')

In [14]:
with open(intents_filepath, 'r') as yml_file:
    content = yaml.safe_load(yml_file)

with open('intents.json', 'w') as json_file:
    json.dump(content, json_file, indent=2)

In [15]:
with open('intents.json', 'r') as infile:
    intents = json.load(infile)

training_sentences = []
training_labels = []
labels = []
responses = []

entity_pattern = re.compile(r'\[(.*?)\]\(subject\)')

utter_greet = "Hello there!"
utter_welcome = "Glad I could be of assistance :)"
utter_goodbye = "Bye-bye!"
utter_im_a_bot = "I'm Explorer AI, a chatbot. I am learning to answer questions about sciencey stuff. Ask away!"
utter_try_something_else = "I don't know how to respond to that. Try something else."
utter_confirmation = {
    'get_explanation': "You'd like to get an explanation of something, is that correct?",
    'affirm': "Noted!",
    'deny': "Got it."
}

for item in intents['nlu']:
    for pattern in item['examples']:
        entity_match = entity_pattern.search(pattern)
        if entity_match:
            subject = entity_match.group(1)
            pattern = entity_pattern.sub(subject, pattern)
        proc_example = process.preprocess(pattern,
                                          stop_words=False,
                                          punctuation=True,
                                          remove_num=True)
        training_sentences.append(proc_example)
        label = item['intent']
        training_labels.append(label)
        if label == 'greet':
            responses.append(utter_greet)
        elif label == 'goodbye':
            responses.append(utter_goodbye)
        elif label == 'thanks':
            responses.append(utter_welcome)
        elif label in ['affirm', 'deny', 'get_explanation']:
            responses.append(utter_confirmation[label])
        elif label == 'bot_challenge':
            responses.append(utter_im_a_bot)
        elif label == 'out_of_scope':
            responses.append(utter_try_something_else)
        if label not in labels:
            labels.append(label)

num_classes = len(labels)

In [16]:
responses = {
    "greet": utter_greet,
    "thanks": utter_welcome,
    "goodbye": utter_goodbye,
    "bot_challenge": utter_im_a_bot,
    "out_of_scope": utter_try_something_else,
    "get_explanation": utter_confirmation["get_explanation"],
    "affirm": utter_confirmation["affirm"],
    "deny": utter_confirmation["deny"],
}

nlu = intents['nlu']
data = []
for item in nlu:
    tag = item['intent']
    item['response'] = responses[tag]
    data.append(item)

intents = {
    "intents": data
}

os.remove('intents.json')
with open('intents.json', 'w') as out_file:
    json.dump(intents, out_file, indent=2)

In [17]:
print('{} training sentenses'.format(len(training_sentences)))
print('{} labels (intents, including OOS)'.format(num_classes))

183 training sentenses
8 labels (intents, including OOS)


In [18]:
encoder = LabelEncoder()
encoder.fit(training_labels)
training_labels = encoder.transform(training_labels)

In [19]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [20]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
____________________________________________________

In [21]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500


2023-01-03 15:44:39.357086: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


6/6 [==============================] - 0s 762us/step - loss: 2.0764 - accuracy: 0.2568
Epoch 2/500
6/6 [==============================] - 0s 657us/step - loss: 2.0710 - accuracy: 0.3060
Epoch 3/500
6/6 [==============================] - 0s 690us/step - loss: 2.0641 - accuracy: 0.3060
Epoch 4/500
6/6 [==============================] - 0s 649us/step - loss: 2.0582 - accuracy: 0.3060
Epoch 5/500
6/6 [==============================] - 0s 664us/step - loss: 2.0513 - accuracy: 0.3060
Epoch 6/500
6/6 [==============================] - 0s 676us/step - loss: 2.0441 - accuracy: 0.3060
Epoch 7/500
6/6 [==============================] - 0s 614us/step - loss: 2.0355 - accuracy: 0.3060
Epoch 8/500
6/6 [==============================] - 0s 719us/step - loss: 2.0260 - accuracy: 0.3060
Epoch 9/500
6/6 [==============================] - 0s 675us/step - loss: 2.0171 - accuracy: 0.3060
Epoch 10/500
6/6 [==============================] - 0s 719us/step - loss: 2.0075 - accuracy: 0.3060
Epoch 11/500
6/6 [===

In [22]:
model.save("chat_model")

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

2023-01-03 15:44:46.907190: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: chat_model/assets
